In [8]:
!pip install transformers torch

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)
CATEGORIES = ["Urgent", "Spam", "Personal", "Work"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def classify_message(message):
    result = classifier(message, CATEGORIES)
    return result["labels"][0]

msg = "Please submit the project by tonight"
print(classify_message(msg))


Urgent


In [9]:
import time

def agent_loop(messages):
    for msg in messages:
        category = classify_message(msg)
        print(f"[AGENT] {msg} → {category}")
        take_action(category)
        time.sleep(1)

def take_action(category):
    if category == "Urgent":
        print("🔔 Notify user")
    elif category == "Spam":
        print("🗑️ Auto-archive")
    elif category == "Work":
        print("📁 Add to work folder")
    else:
        print("📌 Save personal message")

In [ ]:
import imaplib
import email
from google.colab import userdata

def fetch_new_messages():
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(userdata.get('EMAIL_USER'), userdata.get('EMAIL_PASS'))
    mail.select("inbox")

    _, data = mail.search(None, 'UNSEEN')
    messages = []

    for num in data[0].split():
        _, msg_data = mail.fetch(num, '(RFC822)')
        msg = email.message_from_bytes(msg_data[0][1])
        messages.append(msg["subject"])
        mail.store(num, '+FLAGS', '\\Seen')

    mail.logout()
    return messages

for _ in range(5):
    new_messages = fetch_new_messages()
    agent_loop(new_messages)
    time.sleep(60)

In [ ]:
classify_message("Urgent meeting today")